# Topic 14: torch.compile & Performance Optimization

## Learning Objectives

By the end of this notebook, you will:
- Understand how torch.compile works (TorchDynamo + TorchInductor)
- Master regional compilation for transformer layers (PyTorch 2.5+)
- Learn mixed precision training (AMP, float16, bfloat16)
- Profile bottlenecks using torch.profiler
- Measure real speedups with comprehensive benchmarks
- Know when and how to use torch.compile effectively
- Optimize memory usage and throughput

---

## 1. The Big Picture: Why torch.compile?

### The Problem with Eager Execution

**Traditional PyTorch (eager mode)**:
- Executes operations one at a time
- Each operation: Python → C++ → CUDA kernel
- Massive Python overhead for small operations
- No cross-operation optimization

**Example overhead**:
```python
x = x + 1      # Launch kernel 1
x = x * 2      # Launch kernel 2
x = x.relu()   # Launch kernel 3
```
- 3 separate kernel launches
- 3x Python interpreter overhead
- 3x memory reads/writes

### The Compilation Revolution (PyTorch 2.0+)

**torch.compile** transforms your model:
1. **Capture graph**: TorchDynamo traces Python execution
2. **Optimize graph**: Fuse operations, eliminate redundancy
3. **Generate kernels**: TorchInductor creates optimized CUDA code
4. **Execute fast**: Compiled kernels run 30-200% faster

**Same example, compiled**:
```python
x = (x + 1) * 2
x = x.relu()
# → Single fused kernel!
```
- 1 kernel launch
- 1 memory read, 1 write
- No Python overhead

### Why This Matters

**Real-world speedups**:
- 🚀 **Training**: 30-50% faster on transformers
- 🚀 **Inference**: 50-200% faster (especially small batches)
- 🚀 **Memory**: Reduced memory traffic → higher throughput

**Modern LLM usage**:
- ✅ **LLaMA 2/3**: torch.compile for training
- ✅ **Mistral**: Compiled inference pipelines
- ✅ **GPT-4**: Likely uses similar compilation
- ✅ **Your models**: Free speedup with one line!

### How It Works: The Stack

```
Your PyTorch Code
        ↓
  TorchDynamo (captures Python execution as graph)
        ↓
  AOTAutograd (ahead-of-time gradients)
        ↓
  TorchInductor (generates optimized kernels)
        ↓
  Triton/CUDA (low-level execution)
```

**You just write**: `model = torch.compile(model)`

**PyTorch handles**: Everything else!

---

In [ ]:
# Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time
from typing import Optional

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check if torch.compile is available
compile_available = hasattr(torch, 'compile')
print(f"\ntorch.compile available: {compile_available}")
if not compile_available:
    print("⚠️ torch.compile requires PyTorch 2.0+")
    print("   Install with: pip install torch>=2.0.0")

---

## 2. Basic torch.compile Usage

### Compilation Modes

PyTorch offers different compilation modes for different use cases:

1. **`default`**: Balanced speed and compilation time
2. **`reduce-overhead`**: Minimize Python overhead (best for small ops)
3. **`max-autotune`**: Maximum optimization (slow compile, fast runtime)

### Simple Example

In [ ]:
class SimpleModel(nn.Module):
    """Simple model to demonstrate torch.compile"""
    
    def __init__(self, d_model: int):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_model)
        self.linear2 = nn.Linear(d_model, d_model)
        self.linear3 = nn.Linear(d_model, d_model)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Many small operations - perfect for compilation
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x


if compile_available:
    # Create two versions: eager and compiled
    d_model = 512
    model_eager = SimpleModel(d_model).to(device)
    model_compiled = torch.compile(SimpleModel(d_model).to(device))
    
    # Copy weights so they're identical
    model_compiled.load_state_dict(model_eager.state_dict())
    
    # Test forward pass
    x = torch.randn(32, 128, d_model, device=device)
    
    # Eager mode
    out_eager = model_eager(x)
    
    # Compiled mode (first run compiles)
    print("Compiling model... (first run)")
    out_compiled = model_compiled(x)
    
    # Check outputs match
    print(f"\nOutputs match: {torch.allclose(out_eager, out_compiled, atol=1e-5)}")
    print(f"Max difference: {(out_eager - out_compiled).abs().max().item():.2e}")
    print("\n✅ torch.compile produces identical results!")
else:
    print("⚠️ Skipping compile demo - PyTorch 2.0+ required")

### Benchmark: Eager vs Compiled

In [ ]:
def benchmark_model(model, x, num_iters=100, warmup=10):
    """Benchmark model throughput"""
    
    # Warmup
    for _ in range(warmup):
        _ = model(x)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    start = time.time()
    for _ in range(num_iters):
        _ = model(x)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    elapsed = time.time() - start
    return elapsed / num_iters


if compile_available and device.type == 'cuda':
    print("Benchmarking Eager vs Compiled")
    print("="*60)
    
    batch_sizes = [8, 16, 32, 64]
    eager_times = []
    compiled_times = []
    
    for batch_size in batch_sizes:
        x = torch.randn(batch_size, 128, d_model, device=device)
        
        eager_time = benchmark_model(model_eager, x)
        compiled_time = benchmark_model(model_compiled, x)
        
        eager_times.append(eager_time * 1000)  # Convert to ms
        compiled_times.append(compiled_time * 1000)
        
        speedup = eager_time / compiled_time
        print(f"Batch {batch_size:2d}: Eager={eager_time*1000:.2f}ms, "
              f"Compiled={compiled_time*1000:.2f}ms, Speedup={speedup:.2f}x")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Time comparison
    x_pos = np.arange(len(batch_sizes))
    width = 0.35
    
    axes[0].bar(x_pos - width/2, eager_times, width, label='Eager', color='orange', alpha=0.7)
    axes[0].bar(x_pos + width/2, compiled_times, width, label='Compiled', color='blue', alpha=0.7)
    axes[0].set_xlabel('Batch Size', fontsize=12)
    axes[0].set_ylabel('Time (ms)', fontsize=12)
    axes[0].set_title('Eager vs Compiled Execution Time', fontsize=14)
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(batch_sizes)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Speedup
    speedups = [e/c for e, c in zip(eager_times, compiled_times)]
    axes[1].bar(x_pos, speedups, color='green', alpha=0.7)
    axes[1].axhline(y=1.0, color='r', linestyle='--', label='Baseline')
    axes[1].set_xlabel('Batch Size', fontsize=12)
    axes[1].set_ylabel('Speedup (x)', fontsize=12)
    axes[1].set_title('torch.compile Speedup', fontsize=14)
    axes[1].set_xticks(x_pos)
    axes[1].set_xticklabels(batch_sizes)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    avg_speedup = np.mean(speedups)
    print(f"\n💡 Average speedup: {avg_speedup:.2f}x")
    print(f"   Compiled models are {(avg_speedup-1)*100:.0f}% faster!")
else:
    print("⚠️ Skipping benchmark - requires CUDA and PyTorch 2.0+")

---

## 3. Compilation Modes Deep Dive

### Understanding the Modes

Different modes optimize for different scenarios:

| Mode | Compile Time | Runtime Speed | Use Case |
|------|--------------|---------------|----------|
| **default** | Fast | Good | General purpose, development |
| **reduce-overhead** | Medium | Better | Small ops, low latency |
| **max-autotune** | Slow | Best | Production, throughput-critical |

In [ ]:
if compile_available and device.type == 'cuda':
    print("Comparing Compilation Modes")
    print("="*70)
    
    # Create models with different modes
    model_default = torch.compile(SimpleModel(d_model).to(device), mode="default")
    model_reduce = torch.compile(SimpleModel(d_model).to(device), mode="reduce-overhead")
    
    # max-autotune can be very slow to compile, so we'll skip it in this demo
    # model_max = torch.compile(SimpleModel(d_model).to(device), mode="max-autotune")
    
    x = torch.randn(32, 128, d_model, device=device)
    
    # Compile (first run)
    print("\nCompiling models...")
    
    start = time.time()
    _ = model_default(x)
    default_compile_time = time.time() - start
    print(f"  default mode: {default_compile_time:.2f}s")
    
    start = time.time()
    _ = model_reduce(x)
    reduce_compile_time = time.time() - start
    print(f"  reduce-overhead mode: {reduce_compile_time:.2f}s")
    
    # Benchmark runtime
    print("\nBenchmarking runtime...")
    default_time = benchmark_model(model_default, x) * 1000
    reduce_time = benchmark_model(model_reduce, x) * 1000
    
    print(f"\nResults:")
    print(f"  default: {default_time:.2f}ms")
    print(f"  reduce-overhead: {reduce_time:.2f}ms")
    print(f"\n💡 reduce-overhead is {default_time/reduce_time:.2f}x faster for this model")
    print(f"   Use reduce-overhead for inference with many small operations")
else:
    print("⚠️ Skipping mode comparison - requires CUDA and PyTorch 2.0+")

---

## 4. Regional Compilation (PyTorch 2.5+)

### The Problem with Full Model Compilation

**Issue**: Some operations don't compile well
- Dynamic control flow (if/else based on tensor values)
- Python data structures modified during forward
- External function calls

**Solution**: Compile only specific regions!

### Using torch.compiler.region

Mark specific code blocks for compilation:
```python
with torch.compiler.region():
    # This code will be compiled
    x = self.layer1(x)
    x = self.layer2(x)
```

In [ ]:
class TransformerBlock(nn.Module):
    """Transformer block with regional compilation"""
    
    def __init__(self, d_model: int, num_heads: int, d_ff: int):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Attention block
        residual = x
        x = self.norm1(x)
        x, _ = self.attention(x, x, x, need_weights=False)
        x = residual + x
        
        # FFN block (good candidate for compilation)
        residual = x
        x = self.norm2(x)
        x = self.ffn(x)
        x = residual + x
        
        return x


# Demo transformer block
print("Transformer Block Compilation")
print("="*60)

d_model = 512
num_heads = 8
d_ff = 2048

block = TransformerBlock(d_model, num_heads, d_ff).to(device)

if compile_available:
    # Compile the entire block
    block_compiled = torch.compile(block)
    
    x = torch.randn(16, 128, d_model, device=device)
    
    print("Compiling transformer block...")
    out = block_compiled(x)
    print(f"Output shape: {out.shape}")
    print("\n✅ Entire transformer block compiled successfully!")
    print("   torch.compile automatically handles attention and FFN layers.")
else:
    print("⚠️ Skipping transformer compilation - PyTorch 2.0+ required")

---

## 5. Mixed Precision Training

### Why Mixed Precision?

**float32 (full precision)**:
- ✅ High accuracy
- ❌ 4 bytes per number
- ❌ Slower compute

**float16 (half precision)**:
- ✅ 2 bytes per number (2x memory savings)
- ✅ 2-3x faster on modern GPUs
- ❌ Reduced range (overflow risk)
- ❌ Reduced precision (underflow risk)

**bfloat16 (brain float)**:
- ✅ 2 bytes per number
- ✅ Same range as float32 (no overflow)
- ✅ 2-3x faster
- ✅ Better for training stability
- ❌ Requires newer GPUs (A100, H100)

### Automatic Mixed Precision (AMP)

PyTorch automatically:
1. Uses float16/bfloat16 where safe
2. Uses float32 where needed (loss computation)
3. Scales gradients to prevent underflow

**Result**: Fast training with stable convergence!

In [ ]:
def train_with_amp(model, optimizer, x, target, use_amp=False, dtype=torch.float16):
    """Training step with optional AMP"""
    
    if use_amp:
        # Create GradScaler for numerical stability
        scaler = torch.amp.GradScaler('cuda')
        
        # Forward pass in mixed precision
        with torch.amp.autocast('cuda', dtype=dtype):
            output = model(x)
            loss = F.mse_loss(output, target)
        
        # Backward pass with gradient scaling
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    else:
        # Standard float32 training
        output = model(x)
        loss = F.mse_loss(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return loss.item()


if device.type == 'cuda':
    print("Mixed Precision Training Benchmark")
    print("="*70)
    
    # Create model and data
    model_fp32 = TransformerBlock(d_model, num_heads, d_ff).to(device)
    model_fp16 = TransformerBlock(d_model, num_heads, d_ff).to(device)
    model_fp16.load_state_dict(model_fp32.state_dict())
    
    optimizer_fp32 = torch.optim.Adam(model_fp32.parameters(), lr=1e-4)
    optimizer_fp16 = torch.optim.Adam(model_fp16.parameters(), lr=1e-4)
    
    batch_size = 32
    seq_len = 128
    x = torch.randn(batch_size, seq_len, d_model, device=device)
    target = torch.randn(batch_size, seq_len, d_model, device=device)
    
    # Warmup
    for _ in range(5):
        _ = train_with_amp(model_fp32, optimizer_fp32, x, target, use_amp=False)
        _ = train_with_amp(model_fp16, optimizer_fp16, x, target, use_amp=True)
    
    torch.cuda.synchronize()
    
    # Benchmark float32
    start = time.time()
    for _ in range(50):
        _ = train_with_amp(model_fp32, optimizer_fp32, x, target, use_amp=False)
    torch.cuda.synchronize()
    fp32_time = (time.time() - start) / 50 * 1000
    
    # Benchmark float16
    start = time.time()
    for _ in range(50):
        _ = train_with_amp(model_fp16, optimizer_fp16, x, target, use_amp=True)
    torch.cuda.synchronize()
    fp16_time = (time.time() - start) / 50 * 1000
    
    speedup = fp32_time / fp16_time
    
    print(f"Results:")
    print(f"  float32: {fp32_time:.2f} ms/iter")
    print(f"  float16 (AMP): {fp16_time:.2f} ms/iter")
    print(f"  Speedup: {speedup:.2f}x")
    
    # Memory comparison
    torch.cuda.reset_peak_memory_stats()
    _ = train_with_amp(model_fp32, optimizer_fp32, x, target, use_amp=False)
    fp32_mem = torch.cuda.max_memory_allocated() / 1024**2
    
    torch.cuda.reset_peak_memory_stats()
    _ = train_with_amp(model_fp16, optimizer_fp16, x, target, use_amp=True)
    fp16_mem = torch.cuda.max_memory_allocated() / 1024**2
    
    print(f"\nMemory usage:")
    print(f"  float32: {fp32_mem:.1f} MB")
    print(f"  float16 (AMP): {fp16_mem:.1f} MB")
    print(f"  Memory saved: {(1 - fp16_mem/fp32_mem)*100:.1f}%")
    
    print(f"\n💡 AMP provides {speedup:.1f}x speedup and {(1-fp16_mem/fp32_mem)*100:.0f}% memory savings!")
    print("   Use AMP for all training - it's nearly free performance.")
else:
    print("⚠️ Skipping AMP benchmark - requires CUDA")

---

## 6. Profiling with torch.profiler

### Finding Bottlenecks

**torch.profiler** helps you:
- Identify slow operations
- Find memory bottlenecks
- See GPU utilization
- Optimize CUDA kernel usage

In [ ]:
if device.type == 'cuda':
    from torch.profiler import profile, ProfilerActivity, record_function
    
    print("Profiling Transformer Block")
    print("="*70)
    
    model = TransformerBlock(d_model, num_heads, d_ff).to(device)
    x = torch.randn(32, 128, d_model, device=device)
    
    # Profile the model
    with profile(
        activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
        record_shapes=True,
        profile_memory=True,
        with_stack=True
    ) as prof:
        with record_function("model_forward"):
            output = model(x)
    
    # Print profiling results
    print("\nTop 10 operations by CUDA time:")
    print(prof.key_averages().table(
        sort_by="cuda_time_total",
        row_limit=10
    ))
    
    print("\n💡 Profiling insights:")
    print("   - Look for operations with high 'CUDA time'")
    print("   - Focus optimization on top 3-5 operations")
    print("   - Check 'Self CUDA %' to find actual bottlenecks")
else:
    print("⚠️ Skipping profiling - requires CUDA")

---

## 7. Complete Optimization Stack

### Combining All Techniques

Let's build a fully optimized training loop using:
1. torch.compile
2. Mixed precision (AMP)
3. Gradient accumulation
4. Efficient data loading

In [ ]:
class OptimizedTransformer(nn.Module):
    """Fully optimized transformer for training"""
    
    def __init__(
        self,
        d_model: int,
        num_heads: int,
        d_ff: int,
        num_layers: int
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)


def optimized_training_loop(
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    num_steps: int = 100,
    batch_size: int = 32,
    seq_len: int = 128,
    use_compile: bool = True,
    use_amp: bool = True,
    gradient_accumulation_steps: int = 1
):
    """Fully optimized training loop"""
    
    # Compile model if requested
    if use_compile and compile_available:
        print("Compiling model...")
        model = torch.compile(model, mode="reduce-overhead")
    
    # Create GradScaler for AMP
    scaler = torch.amp.GradScaler('cuda') if use_amp and device.type == 'cuda' else None
    
    model.train()
    total_time = 0
    
    for step in range(num_steps):
        step_start = time.time()
        
        # Simulate data loading
        x = torch.randn(batch_size, seq_len, d_model, device=device)
        target = torch.randn(batch_size, seq_len, d_model, device=device)
        
        # Forward pass with AMP
        if use_amp and device.type == 'cuda':
            with torch.amp.autocast('cuda', dtype=torch.float16):
                output = model(x)
                loss = F.mse_loss(output, target)
                loss = loss / gradient_accumulation_steps
            
            # Backward with gradient scaling
            scaler.scale(loss).backward()
            
            # Update weights every N steps
            if (step + 1) % gradient_accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        else:
            # Standard training
            output = model(x)
            loss = F.mse_loss(output, target)
            loss = loss / gradient_accumulation_steps
            
            loss.backward()
            
            if (step + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        step_time = time.time() - step_start
        if step >= 10:  # Skip warmup
            total_time += step_time
        
        if (step + 1) % 25 == 0:
            avg_time = total_time / (step - 9) * 1000 if step > 9 else 0
            print(f"Step {step+1}/{num_steps}: Loss={loss.item():.4f}, "
                  f"Time={step_time*1000:.2f}ms, Avg={avg_time:.2f}ms")
    
    avg_time = total_time / (num_steps - 10)
    return avg_time


if device.type == 'cuda' and compile_available:
    print("\nComprehensive Optimization Benchmark")
    print("="*70)
    
    # Create model
    d_model = 512
    num_heads = 8
    d_ff = 2048
    num_layers = 4
    
    configs = [
        ("Baseline (no opt)", False, False),
        ("AMP only", False, True),
        ("Compile only", True, False),
        ("AMP + Compile", True, True),
    ]
    
    results = []
    
    for name, use_compile, use_amp in configs:
        print(f"\n{name}:")
        print("-" * 70)
        
        model = OptimizedTransformer(d_model, num_heads, d_ff, num_layers).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        
        avg_time = optimized_training_loop(
            model, optimizer,
            num_steps=50,
            use_compile=use_compile,
            use_amp=use_amp
        )
        
        results.append((name, avg_time * 1000))
        print(f"\n✓ Average time: {avg_time*1000:.2f}ms/step")
    
    # Visualize results
    names = [r[0] for r in results]
    times = [r[1] for r in results]
    baseline = times[0]
    speedups = [baseline / t for t in times]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Absolute times
    colors = ['red', 'orange', 'blue', 'green']
    axes[0].barh(names, times, color=colors, alpha=0.7)
    axes[0].set_xlabel('Time (ms/step)', fontsize=12)
    axes[0].set_title('Training Time by Configuration', fontsize=14)
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Plot 2: Speedups
    axes[1].barh(names, speedups, color=colors, alpha=0.7)
    axes[1].axvline(x=1.0, color='r', linestyle='--', label='Baseline')
    axes[1].set_xlabel('Speedup (x)', fontsize=12)
    axes[1].set_title('Speedup vs Baseline', fontsize=14)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*70)
    print("Summary:")
    for i, (name, time) in enumerate(results):
        print(f"  {name}: {time:.2f}ms ({speedups[i]:.2f}x)")
    
    best_speedup = max(speedups)
    print(f"\n💡 Best configuration: {best_speedup:.2f}x faster than baseline!")
    print("   Always use AMP + compile for training.")
else:
    print("⚠️ Skipping comprehensive benchmark - requires CUDA and PyTorch 2.0+")

---

## Mini Exercises

### Exercise 1: Benchmark Custom Model

Create your own model and benchmark eager vs compiled performance.
Include at least 5 layers with different operations (Linear, Conv, ReLU, etc.)

In [ ]:
# Your code here


In [ ]:
# Solution
class MyCustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(512, 512, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(512, 512, kernel_size=3, padding=1)
        self.linear1 = nn.Linear(512, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.norm = nn.LayerNorm(512)
    
    def forward(self, x):
        # x: (batch, seq_len, 512)
        x_t = x.transpose(1, 2)  # (batch, 512, seq_len)
        x_t = F.relu(self.conv1(x_t))
        x_t = F.relu(self.conv2(x_t))
        x = x_t.transpose(1, 2)  # (batch, seq_len, 512)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        x = self.norm(x)
        return x

if compile_available and device.type == 'cuda':
    model_eager = MyCustomModel().to(device)
    model_compiled = torch.compile(MyCustomModel().to(device))
    
    x = torch.randn(32, 128, 512, device=device)
    
    eager_time = benchmark_model(model_eager, x)
    compiled_time = benchmark_model(model_compiled, x)
    
    print(f"Custom Model Benchmark:")
    print(f"  Eager: {eager_time*1000:.2f}ms")
    print(f"  Compiled: {compiled_time*1000:.2f}ms")
    print(f"  Speedup: {eager_time/compiled_time:.2f}x")
else:
    print("⚠️ Requires CUDA and PyTorch 2.0+")

### Exercise 2: Profile a Bottleneck

Create a model with an intentional bottleneck and use torch.profiler to find it.

In [ ]:
# Your code here


In [ ]:
# Solution
class BottleneckModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fast = nn.Linear(512, 512)
        # Intentional bottleneck: very large intermediate
        self.slow = nn.Linear(512, 8192)
        self.output = nn.Linear(8192, 512)
    
    def forward(self, x):
        x = self.fast(x)  # Fast operation
        x = self.slow(x)  # Bottleneck!
        x = F.relu(x)
        x = self.output(x)  # Bottleneck!
        return x

if device.type == 'cuda':
    from torch.profiler import profile, ProfilerActivity
    
    model = BottleneckModel().to(device)
    x = torch.randn(32, 128, 512, device=device)
    
    with profile(activities=[ProfilerActivity.CUDA]) as prof:
        output = model(x)
    
    print("Top operations by CUDA time:")
    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=5))
    print("\n💡 The 8192-dimensional layers are the bottleneck!")
else:
    print("⚠️ Requires CUDA")

### Exercise 3: Compare Data Types

Benchmark float32, float16, and bfloat16 (if available) for the same model.

In [ ]:
# Your code here


In [ ]:
# Solution
if device.type == 'cuda':
    model = TransformerBlock(512, 8, 2048).to(device)
    x = torch.randn(32, 128, 512, device=device)
    target = torch.randn(32, 128, 512, device=device)
    optimizer = torch.optim.Adam(model.parameters())
    
    dtypes = []
    if torch.cuda.is_available():
        dtypes.append(("float32", None))
        dtypes.append(("float16", torch.float16))
        # Check if bfloat16 is supported
        if torch.cuda.is_bf16_supported():
            dtypes.append(("bfloat16", torch.bfloat16))
    
    print("Data Type Comparison:")
    print("="*60)
    
    results = []
    for name, dtype in dtypes:
        # Warmup
        for _ in range(5):
            if dtype:
                with torch.amp.autocast('cuda', dtype=dtype):
                    out = model(x)
            else:
                out = model(x)
        
        # Benchmark
        torch.cuda.synchronize()
        start = time.time()
        for _ in range(50):
            if dtype:
                with torch.amp.autocast('cuda', dtype=dtype):
                    out = model(x)
            else:
                out = model(x)
        torch.cuda.synchronize()
        elapsed = (time.time() - start) / 50 * 1000
        
        results.append((name, elapsed))
        print(f"  {name}: {elapsed:.2f}ms")
    
    baseline = results[0][1]
    print(f"\nSpeedups vs float32:")
    for name, time in results:
        speedup = baseline / time
        print(f"  {name}: {speedup:.2f}x")
else:
    print("⚠️ Requires CUDA")

---

## Key Takeaways

1. **torch.compile is free speedup**: 30-50% faster with one line
2. **Use reduce-overhead mode**: Best for inference and small operations
3. **Mixed precision (AMP) is essential**: 2-3x faster, half the memory
4. **Combine techniques**: AMP + compile = maximum performance
5. **Profile to find bottlenecks**: Don't guess, measure
6. **bfloat16 > float16**: Better numerical stability for training
7. **Compilation takes time**: First run is slow, subsequent runs are fast

## When to Use Each Technique

**torch.compile**:
- ✅ Training large models
- ✅ Inference (especially small batches)
- ✅ Models with many small ops
- ❌ Highly dynamic models (lots of control flow)

**Mixed Precision (AMP)**:
- ✅ Always for training (unless debugging)
- ✅ Inference on GPU
- ✅ Models that fit in memory
- ❌ CPU inference

**Profiling**:
- ✅ When optimizing bottlenecks
- ✅ Before major refactoring
- ✅ Comparing implementations
- ❌ Every training run (overhead)

## Modern LLM Usage (2025)

**LLaMA 2/3**:
- torch.compile for training
- bfloat16 mixed precision
- Regional compilation for custom kernels

**Mistral**:
- Compiled inference pipelines
- float16 for deployment
- Custom Triton kernels for attention

**Your Production Models**:
- Always compile for inference
- Always use AMP for training
- Profile before optimizing
- Use bfloat16 on A100/H100

---

## Next Steps

Continue to: [Topic 15: Production PyTorch Best Practices](15_production_pytorch.ipynb)

---

## Further Reading

- [PyTorch 2.0 Release Notes](https://pytorch.org/get-started/pytorch-2.0/)
- [torch.compile Tutorial](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)
- [Automatic Mixed Precision Guide](https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html)
- [torch.profiler Documentation](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html)
- [TorchInductor Deep Dive](https://dev-discuss.pytorch.org/t/torchinductor-a-pytorch-native-compiler-with-define-by-run-ir-and-symbolic-shapes/747)
- [bfloat16 for Deep Learning](https://arxiv.org/abs/1905.12322)